In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query

In [2]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
from DataPrepare.join_data_preparation import JoinDataPreparator
schema = gen_job_light_imdb_schema('/home/ziniu.wzn/imdb-benchmark')
from DataPrepare.join_data_preparation import JoinDataPreparator
hdf_path = "/home/ziniu.wzn/imdb-benchmark/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

movie_info_idx.movie_id = title.id
movie_info.movie_id = title.id
cast_info.movie_id = title.id
movie_keyword.movie_id = title.id
movie_companies.movie_id = title.id


In [3]:
from Models.pgmpy_BN import Pgmpy_BN, build_meta_info
def train_all(algo="chow-liu", max_parents=1):
    for i, relationship_obj in enumerate(schema.relationships):
        sample_size = 10000000
        relation = relationship_obj.identifier
        df, meta_types, null_values, full_join_est = prep.generate_n_samples(
                        sample_size, relationship_list=[relation], post_sampling_factor=10)
        print(full_join_est)
        print(len(df), len(df.columns))
        print(meta_types)
        meta_info = build_meta_info(df.columns, null_values)
        bn = Pgmpy_BN(relation, meta_info, full_join_est)
        model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl"
        print(model_path)
        bn.build_from_data(df, algorithm=algo, max_parents=max_parents, ignore_cols=['id'], sample_size=200000)
        pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        """
        try:
            BN.model = Bn.model.to_junction_tree()
            model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_junction.pkl"
            pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        except:
            print("This BN is not able to transform into junction tree, probably because it's not connected, just use BN")
        """

In [ ]:
train_all("chow-liu", 3)

In [5]:
from Evaluation.parse_query_imdb import prepare_join_queries

In [11]:
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
ensemble_location = "../Benchmark/IMDB/ensemble_loader.pkl"
#query_filename = "/home/ziniu.wzn/imdb-benchmark/cardinality/job_all.sql"
query_filename = "../Benchmark/IMDB/job-light.sql"
pairwise_rdc_path = "../Benchmark/IMDB/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=False, true_card_exist=True)

In [ ]:
parsed_queries[4]

In [9]:
from Models.BN_ensemble_model import BN_ensemble
bn_ensemble = BN_ensemble(schema)
algo = "chow-liu"
distinct_mapping = dict()
distinct_mapping['movie_keyword.keyword_id']={117: 8, 8200: 10, 398: 5, 7084: 20}
distinct_mapping['movie_companies.company_id']={22956: 62}
for i in range(5):
    with open(f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl", "rb") as f:
        bn = pickle.load(f)
        bn.init_inference_method()
        bn.n_distinct_mapping = distinct_mapping
    bn_ensemble.bns[i] = bn

In [12]:
queries = bn_ensemble.parse_query_all(parsed_queries)

{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 62}} movie_keyword.keyword_id 117.0
None
8
{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 62}} movie_keyword.keyword_id 8200.0
None
10
{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 62}} movie_keyword.keyword_id 8200.0
None
10
{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 62}} movie_keyword.keyword_id 8200.0
None
10
{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 62}} movie_keyword.keyword_id 398.0
None
5
{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 62}} movie_keyword.keyword_id 398.0
None
5
{'movie_keyword.keyword_id': {117: 8, 8200: 10, 398: 5, 7084: 20}, 'movie_companies.company_id': {22956: 6

In [13]:
queries[4]

[6575448.0,
 {'bn_index': 3,
  'inverse': False,
  'expectation': ['title.mul_movie_companies.movie_id'],
  'query': {'movie_keyword.movie_keyword_nn': 0,
   'movie_keyword.keyword_id': [2]},
  'n_distinct': {'movie_keyword.movie_keyword_nn': array([1]),
   'movie_keyword.keyword_id': array([0.57142857])}}]

In [14]:
with open("../Benchmark/IMDB/job-light.sql", "rb") as f:
    real_query = f.readlines()

In [15]:
import time
latency = []
q_error = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        #this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time()-tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred/true[i], true[i]/pred)
    print(real_query[i])
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_error.append(error)

b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2||715\n'
true cardinality 715, predicted 3536.083801292903 with q-error 4.9455717500600045
b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=113 AND mc.company_type_id=2 AND t.production_year>2005 AND t.production_year<2010||9\n'
true cardinality 9, predicted 1.9135314657190519 with q-error 4.703345704648787
b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2 AND t.production_year>2010||47\n'
true cardinality 47, predicted 13.748553492837665 with q-error 3.4185414505231213
b'SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info

b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_companies mc WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=mc.movie_id AND mi_idx.info_type_id=101 AND mi.info_type_id=3 AND t.production_year>2005 AND t.production_year<2008 AND mc.company_type_id=2||75440\n'
true cardinality 75440, predicted 89693.29525679503 with q-error 1.188935515068863
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_companies mc WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=mc.movie_id AND mi_idx.info_type_id=113 AND mi.info_type_id=105||72\n'
true cardinality 72, predicted 132.81843024602645 with q-error 1.8447004200837007
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,movie_companies mc WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=mc.movie_id AND mi_idx.info_type_id=101 AND mi.info_type_id=3 AND t.production_year>2000 AND t.production_year<2010 AND mc.company_type_id=2||305691\n'
true cardinality 305691, predict

b'SELECT COUNT(*) FROM title t,movie_info mi,movie_companies mc,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mc.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND ci.role_id=2 AND mi.info_type_id=16 AND t.production_year>2000||9537310863\n'
true cardinality 9537310863, predicted 2608330148.5945497 with q-error 3.6564814726920223
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND mi.info_type_id=3 AND mi_idx.info_type_id=100||492943940\n'
true cardinality 492943940, predicted 1007723674.8682328 with q-error 2.0442967102267913
b'SELECT COUNT(*) FROM title t,movie_info mi,movie_info_idx mi_idx,cast_info ci,movie_keyword mk WHERE t.id=mi.movie_id AND t.id=mi_idx.movie_id AND t.id=ci.movie_id AND t.id=mk.movie_id AND mi.info_type_id=3 AND mi_idx.info_type_id=100 AND t.production_year>2010||32355583\n'
true cardinality 32355583, pred

In [16]:
for i in [50, 90, 95, 100]:
    print(np.percentile(q_error, i))
print(np.mean(latency))

1.3008735566391971
4.570418987102102
4.830331124184182
19.139993461250405
0.26437131336757114
